In [1]:
import pandas as pd

In [40]:
df_cartola = pd.read_csv('input\\Cartola valorizada.csv')
df_traductor_bodega_sigfe = pd.read_excel('input\\asociacion_bodega_sigfe.xlsx', header = 3)

df_traductor_sigfe_sigcom_cristian = pd.read_excel('input\\Categorías de insumos.xlsx')


df_traductor_bodega_sigfe = df_traductor_bodega_sigfe.rename(columns = {'Nombre Items': \
                                                                        'Item SIGFE'})
TRADUCTOR_ITEM_SIGFE_ITEM_SIGCOM = df_traductor_sigfe_sigcom_cristian.iloc[66:83, [0, 1]]
TRADUCTOR_ITEM_SIGFE_ITEM_SIGCOM = TRADUCTOR_ITEM_SIGFE_ITEM_SIGCOM.rename(columns = 
                                                                {'Destino INT': 'Item SIGFE',
                                                                'Unnamed: 1': 'Item SIGCOM'})

TRADUCTOR_DESTINO_INT_CC_SIGCOM = df_traductor_sigfe_sigcom_cristian.iloc[0:63, [0, 1]]\
                                         .rename(columns = {'Unnamed: 1': 'CC SIGCOM'})

In [41]:
df_traductor_bodega_sigfe

,Nº,Código,Descripción,Familia,Items,Item SIGFE,Prec.Uni.Neto,Stock Min.,Stock Rep.,Stock Opt.,Stock Max.,Stock Actual
0,6287,10,INTERNACION,INSUMOS,2204005003,Otros Insumos clínicos,0,0,0,0,0,NaN
1,5938,20,REMODEL. CENTRAL ESTERILIZACION,MANTENCION Y REPARACION,2206001,Mantenimiento y reparación de edificaciones,0,0,0,0,0,NaN
2,4815,67,PEC EVALUACION LABORATORIO,EXAMENES,221299900201,Compras de Examenes,224934,0,0,0,0,NaN
3,9619,401,AFATINIB 40 MG,FARMACIA,220400400102,Farmacia - Compras Extra Sistema,1159256,0,0,0,0,0.0
4,6833,A006300,ALCOHOL 70º 1 LITRO,LABORATORIO,2204003002,Otros químicos,1149,0,0,0,0,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9444,6354,Z930102,RECARGA DE GRAPADORA LINEAL CORTANTE ENDOSCÓPI...,TORAX,2204005003,Otros Insumos clínicos,0,0,0,0,0,NaN
9445,4642,Z930200,TROCAR VTC Nº5.5,TORAX,2204005003,Otros Insumos clínicos,13479,12,18,25,37,12.0
9446,4643,Z930300,TROCAR VTC Nº11.5,TORAX,2204005003,Otros Insumos clínicos,12500,6,8,11,17,0.0
9447,4644,Z930400,TROCAR STEP 5 MM (AUTOSUTURE),TORAX,2204005003,Otros Insumos clínicos,16392,0,1,1,1,0.0


In [42]:
unidas = pd.merge(df_cartola, df_traductor_bodega_sigfe, how = 'left', left_on = 'Codigo Articulo',
                                                                        right_on = 'Código')

In [44]:
unidas_concepto_presup = pd.merge(unidas, TRADUCTOR_ITEM_SIGFE_ITEM_SIGCOM, how = 'left',
                                                                            on = 'Item SIGFE')

In [33]:
unidas_concepto_presup

Instrumental Quirúrgico                                                            5820
Farmacia                                                                           3309
Prótesis                                                                            779
Materiales de Oficina                                                               774
BIENES Y SERVICIOS DE CONSUMO: Para Personas                                        745
Otros Químicos                                                                      388
Otros Materiales Y Utiles De Aseo                                                   376
Menaje para Oficina, Casino y Otros                                                 235
Insumos, Repuestos y Accesorios Computacionales                                      92
Otros Materiales, Repuestos y Útiles Diversos para Mantenimiento y Reparaciones      63
Vest Y Acces P Diversas Para El Personal                                             22
Equipos Menores                 

In [32]:
TRADUCTOR_ITEM_SIGFE_ITEM_SIGCOM['Item SIGFE']

66                                      Equipos menores
67      Insumos, repuestos y accesorios computacionales
68    Mantenimiento y Reparacion de Maquinas y Equip...
69    Mantenimiento y Reparación de Otras Maquinaria...
70                              Material de Laboratorio
71                                Materiales de oficina
72                      Materiales y útiles quirúrgicos
73                  Menaje para oficina, casino y otros
74                               Otros Insumos clínicos
75                    Otros materiales y útiles de aseo
76    Otros materiales, repuestos y útiles diversos ...
77                                       Otros químicos
78                      Otros textiles act hospitalaria
79                             Oxigeno y gases clínicos
80                              Para personas-Pacientes
81                                             Prótesis
82       Otros Insumos clínicos - Compras Intermediadas
Name: Item SIGFE, dtype: object